In [ ]:
import importlib
import sys
from pathlib import Path
from typing import Dict, Union

import msticpy.context.tiproviders
import msticpy.context.contextproviders
import msticpy.data.core
from msticpy.context.tiproviders.ti_provider_base import TIProvider
from msticpy.context.contextproviders.context_provider_base import ContextProvider

from msticpy.data import drivers
from msticpy.context.lookup import Lookup
from msticpy.data.drivers import DriverBase

# Here - let's return the object that needs
# to have the custom drivers assigned - e.g. data.core.__init__
# or content.tiproviders, context.contextproviders
_PLUGIN_TYPES = {
    DriverBase: drivers,
    TIProvider: msticpy.context.tiproviders,
    ContextProvider: msticpy.context.contextproviders,
}


def load_plugins(plugin_path: Union[str, Path]):
    sys.path.append(plugin_path)
    for module_file in Path(plugin_path).glob("*.py"):
        module = importlib.import_module(module_file.stem)
        for obj in vars(module).values():
            if not isinstance(obj, type):
                continue
            plugin_type, reg_object = _get_plugin_type(obj)
            if plugin_type:
                # if no specified registration, use the parent class
                reg_object = reg_object or plugin_type
                reg_object.CUSTOM_PROVIDERS[obj.__name__.casefold()] = obj


def _get_plugin_type(plugin_class):
    return next(
        (
            (parent_class, dest)
            for parent_class, dest in _PLUGIN_TYPES.items()
            if issubclass(plugin_class, parent_class)
        ),
        (None, None),
    )


```yaml
PluginPaths:
  - path1
  - path2
TIProviders:
  OTX:
    Args:
      AuthKey: 1234567890
    Primary: True
    Provider: "OTX"
  XYZ:
    Args:
      AuthKey: 1234567890
    Primary: True
    ProviderDef:
      Path: /home/ian/myti.yaml
      Entry: TIProv1
    
  

In [ ]:
# on package config load:
from msticpy.common.pkg_config import get_config

# plugin_folders = get_config("Plugins.modules") # list
plugin_folders = [plugin_path]
detected_plugins: Dict[type, Dict[str, type]] = defaultdict(dict)

for plugin_folder in plugin_folders:
    detected_plugins.update(load_plugins(plugin_folder))

for plugin_type, plugins in detected_plugins.items():
    if not hasattr(plugin_type, "CUSTOM_PLUGINS"):
        plugin_type.CUSTOM_PLUGINS = {}
    plugin_type.CUSTOM_PLUGINS.update(plugins)

HttpTIProvider.CUSTOM_PLUGINS


{'custom': custom_ti.Custom}